# Elguja Dadunashvili: Machine-Learning-Verfahren für die typologisch-vergleichenden Märchenforschung - Teil 2

## Modul für die Aggregation des Korpus von unbekannten Daten 

[Zelle # 1]

1-5: Der vorhandene Daten-Korpus löschen (falls vorhanden). Die Funktion wird im Folgenden zwecks Beseitigung von weiteren Aggregationen verwendet.

6-61: Aggregation des Rohdatenkorpus.

62-70: Aggregation der "episode_string_such.csv" Datei.

71-76: 'episode_string_such.csv' laden, von den überflüssigen Zeichen bereinigen, fehlende Feldwerte auffüllen und in 'episode_binar_such.csv' umwandeln.

77-78: Unnotige Aggregationen werden gelöscht.

In [ ]:
def entfernen(var):
    import os
    if os.path.exists(var):
        os.remove(var)
entfernen('episode_binar_roh.csv')
import glob
def word_to_lex(word):
    ret=(word) 
    return ret
def write_back(words):
    with open("episode_temp_roh.csv", "a", encoding='utf-8') as output:      
        for word in words:
            as_lex = word_to_lex(word[0])
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)
def clean(line):
    line = line.replace("\n"," ").strip()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe")
    line = line.replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","")
    line = line.replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a")
    line = line.replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ")
    line = line.replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ")
    line = line.replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','')
    line = line.replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    for i, _ in enumerate(line):
        if (i !=0) and (i!=2):
            line[i]=line[i].lower()
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret
with open("episode_temp_roh.csv", "w", encoding='utf-8') as output:
    output.write ("quelle,episode,index_string,index_binar\n")
pairs = []
for file in glob.glob("Datendepot/*.txt"):
    with open(file, 'r', encoding='utf-8', errors='ignore') as episode:
        for line in episode.readlines():
            clean_words = clean(line)
            pairs = pairs + clean_words
write_back(pairs)
from collections import Counter
fin = open("episode_temp_roh.csv",'r', encoding ='utf-8')
fout = open('episode_string_roh.csv', "wt", encoding ='utf-8')
for efz in fin:
    fout.write(efz.replace(', "["',"").replace('"[', "")
               .replace(']"',"").replace("', '", "','")
               .replace(" '", "'"))   
fin.close()
fout.close()
fin = open('episode_string_roh.csv','r', encoding='utf-8') 
fout = open('episode_binar_roh.csv', "wt", encoding='utf-8')
for efz in fin:
    fout.write(efz.replace("'","").replace(',,0,0',',test,0,0')) #dabei wird leergebliebene Datenfeld "episode" mit dem Wert 'test' aufgefüllt        
fin.close()
fout.close()
entfernen('episode_temp_roh.csv')
entfernen('episode_string_roh.csv')

## Modul für die Suche nach den Trainingsdaten

[Zelle # 2]

1-11: Rohdatenkopus aus dem Datenfeld 'epsode' bestimmen und alle andere Felder ignorieren.

12-17: Datenfeld 'episode' von Stoppwörter bereinigen, zur Auffüllung der fehlende Werte eingesetzte synthetischer Stoff (hier das Wort 'test') entfernen.  

In [ ]:
import csv
import pandas as pd
df = pd.read_csv('episode_binar_roh.csv', encoding='utf-8')
t_corpus = []
file = 'episode_binar_roh.csv'
with open(file, "r", encoding='utf-8') as maerchen:
    maerchen = df.episode
    reader = csv.reader(maerchen, delimiter = ',')
    for row in reader:
        lst = str(row)
        t_corpus.append(lst)
custom_stop_word_list=['test']
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_stop_words_list=stopwords.words('german')
de = custom_stop_word_list + NLTK_stop_words_list

[Zelle # 3]

1-14: Vektorisieren, Transformieren und die Metrik für die Vorhersagefunktion definieren

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer
vectorizers = vectorizer(analyzer='word', ngram_range=(1,1), 
                         min_df = 2, stop_words = de)
vokabular =  vectorizers.fit_transform(t_corpus)
metrik = cosine_similarity
def find_similar(vokabular, index, top_n =-1):   
    metriks = metrik(vokabular[index:index+1], vokabular).flatten()
    related_docs_indices = [i for i in metriks.argsort()[::-1] 
                            if i != index]
    return [(index, metriks[index]) for index 
            in related_docs_indices][0:top_n]

## Nach den inhaltlich ähnlichen Datensätzen suchen

[Zelle # 4]

1-3: Gesuchte Episode (Zielepisode) eingeben und einblenden lassen.

Anmerkung: Die gesuchte Episode wird über die 'n_te'-Variable eingegeben (manuell eingepflegte Zielepisoden sollen deshalb immer in der ersten Reihen der Korpusdatensätzen stehen und alphanummerisch angeordnet sein. Wir bevorzugen, dass diese Daten mit dem Prefix '!_' + Kleinbuchstabe benannt wird.)


In [ ]:
n_te = 0
synt = t_corpus[n_te]
print(df.quelle[n_te],'-', synt)

[Zelle # 5]

1-3: Suchen starten und die Ergebnisse einblenden lassen

In [ ]:
for index, score in (find_similar(vokabular, n_te)):
        print(index,'-', score,'-', df.quelle[index],'-', 
           t_corpus[index],'-', df.index_string[index],'-', df.index_binar[index],'\n' )

## Ende 